## Generate images for evaluation

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import sys
utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import torch
from diffusers import DiffusionPipeline
from utils.dataset_info import get_subjects_prompts_info


# Single Subject Generation
single_subject = []                            # "backpack"
# Single Prompt Generation
single_prompt = []                            # e.g. ["a {0} {1} near the pool"]

num_generation = 4


output_path = "../../outputs/subjects/custom_diffusion"
subjects = os.listdir("../../logs/subjects/custom_diffusion/")
dataset_info_path = "../../pcs_dataset/info.json"
os.makedirs(output_path, exist_ok=True)

prompts_info = get_subjects_prompts_info(dataset_info_path)

if len(single_subject):
    subjects = single_subject

for subject in subjects:

    print(f"***** Subject: {subject} *****")
    
    pipeline = DiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16).to("cuda")
    pipeline.unet.load_attn_procs(f"../../logs/subjects/custom_diffusion/{subject}", weight_name="pytorch_custom_diffusion_weights.bin")
    pipeline.load_textual_inversion(f"../../logs/subjects/custom_diffusion/{subject}", weight_name="<new1>.bin")

    os.makedirs(os.path.join(output_path, subject), exist_ok=True)

    if len(single_prompt):
        prompts = single_prompt
    else:
        prompts = prompts_info[subject]["prompts"]

    for prompt in prompts:
        if "a {0} {1}" in prompt:
            prompt = prompt.replace("a {0}", "<new1>")
        
        else:
            prompt = prompt.replace("{0}", "<new1>")
        prompt = prompt.replace("{1}", prompts_info[subject]['class'])
        
        print(f"**Prompt**: {prompt}")

        if prompt not in os.listdir(os.path.join(output_path, subject)):

            image = pipeline(prompt, num_inference_steps=50, num_images_per_prompt=num_generation, guidance_scale=7.5).images

            save_path = os.path.join(output_path, subject, f"{prompt}")
            os.makedirs(save_path, exist_ok=True)

            for img_idx, img in enumerate(image):
                img.save(os.path.join(save_path, f"{img_idx:04d}.jpg"))
    
    print(f"Finished custom diffusion in subject: {subject}!")

## Evaluation

### Calculate the similarity for each sample
Calculate img to img similarity and text to img similarity by CLIP Evaluator

In [1]:
import os, sys, json

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import numpy as np
from utils.clip_eval import evaluate_i2i, evaluate_t2i

In [2]:
def convert_to_native(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, np.generic):
        return data.item()
    elif isinstance(data, dict):
        return {key: convert_to_native(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_native(item) for item in data]
    else:
        return data
    
def add_evaluation(file_path, new_data):
    # check whether the file exist
    if not os.path.exists(file_path):
        with open(file_path, 'w') as file:
            json.dump(convert_to_native(new_data), file, indent=4)
    else:
        with open(file_path, 'r') as file:
            data = json.load(file)
        data.update(convert_to_native(new_data))
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)


In [ ]:
outputs_path = "../../outputs/subjects/custom_diffusion"
eval_res_path = "../../eval_results/subjects/custom_diffusion"
dataset_path = "../../pcs_dataset/subjects"
dataset_info_path = "../../pcs_dataset/info.json"
os.makedirs(eval_res_path, exist_ok=True)

subjects_list = os.listdir(outputs_path)

for subject in subjects_list:
    evaluation_res = dict()
    print(f"***** Subject: {subject} *****")

    res_for_each_subject =dict()

    for prompt in os.listdir(os.path.join(outputs_path, subject)):
        print(f"**Prompt**: {prompt}")

        res_for_each_prompt =dict()

        for generate_img_name in os.listdir(os.path.join(outputs_path, subject, prompt)):
            generate_img_path = os.path.join(outputs_path, subject, prompt, generate_img_name)
            res_for_each_prompt[generate_img_name] = [evaluate_i2i(generate_img_path, os.path.join(dataset_path, subject)), evaluate_t2i(generate_img_path, prompt)]
        
        res_for_each_subject[prompt] = res_for_each_prompt
        print(res_for_each_prompt)

    evaluation_res[subject] = res_for_each_subject

    add_evaluation(os.path.join(eval_res_path, "evaluation_results.json"), evaluation_res)

### Calculate the average similarity

In [5]:
import json

eval_res_path = "../../eval_results/subjects/custom_diffusion/evaluation_results.json"

with open(eval_res_path, "r") as f:
    data = json.load(f)

img_sim = 0.0
text_sim = 0.0
cnt = 0

for subject in data:
    for prompt in data[subject]:
        for sample in data[subject][prompt]:
            img_sim = img_sim + data[subject][prompt][sample][0]
            text_sim = text_sim + data[subject][prompt][sample][1]
            cnt = cnt + 1
print("Image Similarity: ", img_sim/cnt, "\nText Similarity:", text_sim/cnt)

Image Similarity:  0.7222560211674278 
Text Similarity: 0.30712475772621073
